In [37]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import algo
import attack

import torch
import os
import json
import random



In [33]:
# data = pd.read_csv('../datasets.nosync/iris.data', header=None)
data = pd.read_csv('../DP-UTIL.nosync/loan_preprocessed.csv', index_col=0)


In [22]:
#Iris
labels = data[4].unique()
for i,l in enumerate(labels):
    data = data.replace(l, i)
data = data.sample(frac=1, random_state=1)
x_target_train = np.array(data[:100].drop(4, axis=1))
y_target_train = np.array(data[:100][4])
x_target_test = np.array(data[100:].drop(4, axis=1))
y_target_test = np.array(data[100:][4])

In [46]:
#Loan
y = data['grade']
X = data.drop('grade', axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X[0:100000], y[0:100000], train_size=0.5, random_state=0)
x_target_train = np.array(X_train)
y_target_train = np.array(y_train)
x_target_test = np.array(X_test)
y_target_test = np.array(y_test)

In [62]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.01
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 10
model.epsilon        = 0

In [63]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

The accuracy of the model : 88.6 %
The accuracy of the model : 88.1 %


0.88142

In [52]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.01
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = True
model.L              = 5
model.epsilon        = 100

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

DP-SGD with sampling rate = 0.01% and noise_multiplier = 0.28762281181154964 iterated over 1000000 steps satisfies differential privacy with eps = 100 and delta = 1e-05.
The accuracy of the model : 82.19999999999999 %
The accuracy of the model : 81.39999999999999 %


0.81396

In [59]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.01
model.max_iter       = 300
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = True
model.L              = 5
model.epsilon        = 100000

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

DP-SGD with sampling rate = 0.01% and noise_multiplier = 0.15221822151677022 iterated over 3000000 steps satisfies differential privacy with eps = 1e+05 and delta = 1e-05.
The accuracy of the model : 86.3 %
The accuracy of the model : 85.7 %


0.85688

In [61]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y_target_test))
model.alpha          = 0.01
model.max_iter       = 200
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = True
model.L              = 10
model.epsilon        = 10000

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

DP-SGD with sampling rate = 0.02% and noise_multiplier = 0.181578173965137 iterated over 1000000 steps satisfies differential privacy with eps = 1e+04 and delta = 1e-05.
The accuracy of the model : 83.8 %
The accuracy of the model : 83.0 %


0.83046

In [130]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(data[4].unique())
model.alpha          = 0.01
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = True
model.L              = 10
model.epsilon        = 0.5

In [15]:
raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X_raw =raw_data.drop('63', axis=1)
y_raw =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

Dataset:  ../datasets/dataset_purchase
Classes in classification task:  100


In [16]:
X_train, x_shadow, y_train, y_shadow = train_test_split(X_raw, y_raw, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")        


(39464, 600) (157859, 600)


/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_28431/3339399080.py:8: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_target_train = np.array(y_train[:X_train_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_28431/3339399080.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])


In [18]:
rand_seed=2
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

model = algo.LogisticRegression_DPSGD()

model.n_classes      = n_classes
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = True
model.L              = 1000
model.epsilon        = 100000

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)


DP-SGD with sampling rate = 10% and noise_multiplier = 0.07487938508493773 iterated over 1000 steps satisfies differential privacy with eps = 1e+05 and delta = 1e-05.


In [ ]:
model.train_acc = model.evaluate(x_target_train, y_target_train, acc=True)
model.test_acc = model.evaluate(x_target_test, y_target_test, acc=True)

# MI

In [64]:
path = 'mia'
attack_models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            attack_models[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            attack_models[file] = torch.load(r+'/'+file)      

NameError: name 'Net_attack' is not defined

In [ ]:
target_model = model

In [ ]:
test_mi_attack(attack_models, target_model, x_target_train, y_target_train, x_target_test, y_target_test)